In [14]:
import sys
import torch
from tqdm import tqdm
from neuralpredictors.measures.np_functions import corr, fev
sys.path.append('/home/charon/project/sensorium/')
sys.path.append('/home/charon/project/sensorium/run')
from shallowmind.api.infer import prepare_inference
from shallowmind.api.eval import test

In [38]:
cfg = '/data2/charon/sensorium/sensorium_baseline/sensorium_baseline.py'
ckpt = '/data2/charon/sensorium/sensorium_baseline/ckpts/exp_name=sensorium_baseline-cfg=sensorium_baseline-val_correlation=0.299.ckpt'
di, mi = prepare_inference(cfg, ckpt)
# hack to make it work with a subject different from config file
# di.hparams.data.test.feature_dir = 'static23656-14-22-GrayImageNet-94c6ff995dac583098847cfecd43e7b6.zip'
di.setup()
dl = di.val_dataloader()

/home/charon/anaconda3/envs/sensorium/lib/python3.9/site-packages/neuralpredictors/layers/readouts/base.py:72: UserWarning: Use of 'gamma_readout' is deprecated. Please consider using the readout's feature-regularization parameter instead
  warnings.warn(


In [39]:
# single trial correlation
preds = []
labels = []
mi.eval()
for i, (x, y) in tqdm(enumerate(dl), total=len(dl)):
    pred = mi.model.forward_test(x)['output']
    preds.append(pred)
    labels.append(y)
preds = torch.cat(preds, dim=0).detach().cpu().numpy()
labels = torch.cat(labels, dim=0).detach().cpu().numpy()
corr(labels, preds, axis=0).mean()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:02<00:00,  1.63it/s]


0.29851946

In [40]:
# Average correlation
import numpy as np
from collections import defaultdict
preds = defaultdict(list)
labels = defaultdict(list)
corrs = []
mi.eval()
for i, (x, y) in tqdm(enumerate(dl), total=len(dl)):
    pred = mi.model.forward_test(x)['output']
    preds[x['frame_image_id']].append(pred)
    labels[x['frame_image_id']].append(y)
for k, v in preds.items():
    preds[k] = torch.cat(v, dim=0).detach().cpu().numpy()
    labels[k] = torch.cat(labels[k], dim=0).detach().cpu().numpy()
    corrs.append(corr(labels[k], preds[k], axis=0).mean())
corrs = np.mean(corrs)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:02<00:00,  1.64it/s]
